In [68]:
import pandas as pd
import requests
import os
from datetime import datetime, date, timedelta
import matplotlib.pyplot as plt
import yfinance as yf
import numpy as np

# Fred API Calls

In [69]:
FRED_API_KEY = os.environ["FRED_API_KEY"]

In [70]:
def fetch_fred_data(series_id, start_date, end_date, frequency, aggregation_method):
   
    # FRED API URL
    url = f'https://api.stlouisfed.org/fred/series/observations'

    # Parameters for the API request
    params = {
        'series_id': series_id,
        'api_key': FRED_API_KEY,
        'file_type': 'json',
        'frequency': frequency,
        'aggregation_method': aggregation_method,
        'observation_start': start_date,
        'observation_end': end_date,
    }

    # Sending the request to the FRED API
    response = requests.get(url, params=params)
    data = response.json()

    # Check for errors
    if 'observations' not in data:
        print(f"Error fetching data for {series_id}: {data.get('error_message', 'Unknown error')}")
        return pd.DataFrame()

    # Extract observations
    observations = data['observations']
    
    # Convert to DataFrame
    df = pd.DataFrame(observations)
    df['date'] = pd.to_datetime(df['date'])
    df['value'] = pd.to_numeric(df['value'], errors='coerce')  # Convert values to numeric

    # Select relevant columns
    df = df[['date', 'value']]
    
    return df

In [71]:
#Define Date Range
end_date = date.today().replace(day=1) - timedelta(days=1)
start_date = (end_date - timedelta(days=10*365.24)).replace(day=1)
print(start_date, end_date)

2014-10-01 2024-09-30


In [72]:
#Fetch Unemployment Rate (UNRATE) data
unemployment_series_id = 'UNRATE' 

# Fetch data
fetched_unemployment_data = fetch_fred_data(unemployment_series_id, start_date, end_date,"m",'avg')


In [73]:
#Fetch Fed Rate (FEDFUNDS) data
fed_series_id = 'FEDFUNDS' 

# Fetch data
fetched_fed_data = fetch_fred_data(fed_series_id, start_date, end_date,"m",'avg')


In [74]:
#Fetch Consumer Price Index for All Urban Consumers: All Items in U.S. City Average (CPIAUSCL) data from 2020-01-01 to 2024-01-01
CPI_series_id = 'CPIAUCSL'

# Fetch data
fetched_cpi_data = fetch_fred_data(CPI_series_id, start_date, end_date,"m",'avg')

In [75]:
#Fetch 3 Month Treasury Bill Rates
threemonth_tbill_series_id = 'TB3MS'

# Fetch data
fetched_threemonth_tbill_data = fetch_fred_data(threemonth_tbill_series_id, start_date, end_date,"m",'avg')

In [76]:
#Fetch 6 Month Treasury Bill Rates
sixmonth_tbill_series_id = 'TB6MS'

# Fetch data
fetched_sixmonth_tbill_data = fetch_fred_data(sixmonth_tbill_series_id, start_date, end_date,"m",'avg')

In [77]:
#Fetch 10 Year Treasury Bill Rates
tenyear_treasury_series_id = 'DGS10'

# Fetch data
fetched_tenyear_treasury_data = fetch_fred_data(tenyear_treasury_series_id, start_date, end_date,"m",'avg')

In [78]:
#Fetch BBB ICE BofA Index Effective Yield
BBB_series_id = 'BAMLC0A4CBBBEY'

# Fetch data
fetched_BBB_data = fetch_fred_data(BBB_series_id, start_date, end_date,"m",'avg')

In [79]:
#Fetch Moody's AAA Corporate Bond Yields
AAA_series_id = 'AAA'

#Fetch data
fetched_AAA_data = fetch_fred_data(AAA_series_id, start_date, end_date,"m",'avg')

In [80]:
#Fetch Moody's BAA Corporate Bond Yields
BAA_series_id = 'BAA'

#Fetch data
fetched_BAA_data = fetch_fred_data(BAA_series_id, start_date, end_date,"m",'avg')

In [81]:
#Fetch SP500 Index
sp500_series_id = 'SP500'

# Fetch data
fetched_sp500_data = fetch_fred_data(sp500_series_id, start_date, end_date,"d",'eop')

In [82]:
#Fetch VIX Data
vix_series_id = 'VIXCLS'

#Fetch historical market data
fetched_vix_data = fetch_fred_data(vix_series_id, start_date, end_date,"d",'eop')


In [83]:
#Fetch GDP Data
gdp_series_id = 'GDP'

#Fetch historical GDP data
fetched_gdp_data = fetch_fred_data(gdp_series_id, start_date, end_date,"q","avg")


In [84]:
#Fetch Initial Jobless Claims Data
initial_claims_series_id = 'ICSA'

#Fetch historical Initial Claims Data
fetched_initial_claims_data = fetch_fred_data(initial_claims_series_id,start_date,end_date,"m","sum")

In [85]:
#Fetch JPY/USD Exchange Rate
jpy_usd_exchange_rate_series_id = 'DEXJPUS'

#Fetch JPY/USD Historical Data
fetched_jpy_usd_exchange_rate_data = fetch_fred_data(jpy_usd_exchange_rate_series_id,start_date,end_date,"m","avg")

In [86]:
#Fetch Non-Farm Payrolls
nonfarm_payroll_series_id = 'PAYEMS'

fetched_nonfarm_payroll_data = fetch_fred_data(nonfarm_payroll_series_id,start_date,end_date,"m","avg")

In [87]:
#Fetch Oil Prices
oilprices_series_id = 'WTISPLC'

#Fetch Oil Prices Historical Data
fetched_oilprices_data = fetch_fred_data(oilprices_series_id,start_date,end_date,"m","avg")

In [88]:
#Data Cleaning
unemployment_data = fetched_unemployment_data.set_index('date')
unemployment_data.index = pd.to_datetime(unemployment_data.index)
unemployment_data = unemployment_data.rename(columns={'value': 'Unemployment Rate'})

fed_data = fetched_fed_data.set_index('date')
fed_data.index = pd.to_datetime(fed_data.index)
fed_data = fed_data.rename(columns={'value': 'Fed Rate'})

cpi_data = fetched_cpi_data.set_index('date')
cpi_data.index = pd.to_datetime(cpi_data.index)
cpi_data = cpi_data.rename(columns={'value': 'CPI'})
last_cpi_value = cpi_data['CPI'].iloc[-1]
last_cpi_date = cpi_data.index[-1]
new_date_range = pd.date_range(start=last_cpi_date + pd.DateOffset(months = 1), end=end_date + pd.DateOffset(months = 1), freq='ME')
extended_df = pd.DataFrame({'CPI': [last_cpi_value] * len(new_date_range)}, index=new_date_range)
extended_df.index.name = 'date'
cpi_data = cpi_data.resample('ME').ffill()
cpi_data = pd.concat([cpi_data, extended_df])
cpi_data.index = cpi_data.index.to_period('M').to_timestamp()

threemonth_tbill_data = fetched_threemonth_tbill_data.set_index('date')
threemonth_tbill_data.index = pd.to_datetime(threemonth_tbill_data.index)
threemonth_tbill_data = threemonth_tbill_data.rename(columns={'value': 'Three Month T Bill Rate'})

sixmonth_tbill_data = fetched_sixmonth_tbill_data.set_index('date')
sixmonth_tbill_data.index = pd.to_datetime(sixmonth_tbill_data.index)
sixmonth_tbill_data = sixmonth_tbill_data.rename(columns={'value': 'Six Month T Bill Rate'})

tenyear_treasury_data = fetched_tenyear_treasury_data.set_index('date')
tenyear_treasury_data.index = pd.to_datetime(tenyear_treasury_data.index)
tenyear_treasury_data = tenyear_treasury_data.rename(columns={'value': '10-Year Treasury Constant Maturity Rate'})

BBB_data = fetched_BBB_data.set_index('date')
BBB_data.index = pd.to_datetime(BBB_data.index)
BBB_data = BBB_data.rename(columns={'value': 'BBB Rate'})

AAA_data = fetched_AAA_data.set_index('date')
AAA_data.index = pd.to_datetime(AAA_data.index)
AAA_data = AAA_data.rename(columns={'value': 'AAA Rate'})

BAA_data = fetched_BAA_data.set_index('date')
BAA_data.index = pd.to_datetime(BAA_data.index)
BAA_data = BAA_data.rename(columns={'value': 'BAA Rate'})

sp500_data = fetched_sp500_data.set_index('date')
sp500_data.index = pd.to_datetime(sp500_data.index)
sp500_data = sp500_data.rename(columns={'value': 'SP500 Index'})
sp500_data = sp500_data.resample('ME').median()
###sp500_data = sp500_data.diff()
sp500_data.index = sp500_data.index.to_period('M').to_timestamp()

VIX_data = fetched_vix_data.set_index('date')
VIX_data.index = pd.to_datetime(VIX_data.index)
VIX_data = VIX_data.rename(columns={'value': 'VIX'})
VIX_data = VIX_data.resample('ME').median()
VIX_data.index = VIX_data.index.to_period('M').to_timestamp()

gdp_data = fetched_gdp_data.set_index('date')
gdp_data.index = pd.to_datetime(gdp_data.index)
gdp_data = gdp_data.rename(columns={'value': 'GDP'})
last_gdp_value = gdp_data['GDP'].iloc[-1]
last_gdp_date = gdp_data.index[-1]
new_date_range = pd.date_range(start=last_gdp_date + pd.DateOffset(months = 1), end=end_date + pd.DateOffset(months = 1), freq='ME')
extended_df = pd.DataFrame({'GDP': [last_gdp_value] * len(new_date_range)}, index=new_date_range)
extended_df.index.name = 'date'
gdp_data = gdp_data.resample('ME').ffill()
gdp_data = pd.concat([gdp_data, extended_df])
gdp_data.index = gdp_data.index.to_period('M').to_timestamp()

initial_claims_data = fetched_initial_claims_data.set_index('date')
initial_claims_data.index = pd.to_datetime(initial_claims_data.index)
initial_claims_data = initial_claims_data.rename(columns={'value': 'Initial Claims'})

jpy_usd_exchange_rate_data = fetched_jpy_usd_exchange_rate_data.set_index('date')
jpy_usd_exchange_rate_data.index = pd.to_datetime(jpy_usd_exchange_rate_data.index)
jpy_usd_exchange_rate_data = jpy_usd_exchange_rate_data.rename(columns={'value': 'JPY/USD Spot Rate'})

nonfarm_payroll_data = fetched_nonfarm_payroll_data.set_index('date')
nonfarm_payroll_data.index = pd.to_datetime(nonfarm_payroll_data.index)
nonfarm_payroll_data = nonfarm_payroll_data.rename(columns={'value': 'Total Nonfarm Employees'})

oilprices_data = fetched_oilprices_data.set_index('date')
oilprices_data.index = pd.to_datetime(oilprices_data.index)
oilprices_data = oilprices_data.rename(columns={'value': 'Spot Crude Oil Price'})

In [89]:
cpi_data

,CPI
date,
2014-10-01,237.430
2014-11-01,236.983
2014-12-01,236.252
2015-01-01,234.747
2015-02-01,235.342
...,...
2024-05-01,313.225
2024-06-01,313.049
2024-07-01,313.534


In [90]:
# Merge dataframes together one by one
merged_data = pd.merge(unemployment_data, cpi_data, on='date')
merged_data = pd.merge(merged_data, threemonth_tbill_data, on='date')
merged_data = pd.merge(merged_data, sixmonth_tbill_data, on='date')
merged_data = pd.merge(merged_data, tenyear_treasury_data, on='date')
merged_data = pd.merge(merged_data, fed_data, on='date')
merged_data = pd.merge(merged_data, BBB_data, on='date')
merged_data = pd.merge(merged_data, sp500_data, on='date')
merged_data = pd.merge(merged_data, VIX_data, on='date')
merged_data = pd.merge(merged_data, gdp_data, on='date')
merged_data = pd.merge(merged_data, initial_claims_data, on='date')
merged_data = pd.merge(merged_data, jpy_usd_exchange_rate_data, on='date')
merged_data = pd.merge(merged_data, nonfarm_payroll_data, on='date')
merged_data = pd.merge(merged_data, oilprices_data, on='date')
merged_data.index.name = "Date"


In [91]:
merged_data.to_csv("VIX_FRED_data.csv")

In [92]:
merged_data

,Unemployment Rate,CPI,Three Month T Bill Rate,Six Month T Bill Rate,10-Year Treasury Constant Maturity Rate,Fed Rate,BBB Rate,SP500 Index,VIX,GDP,Initial Claims,JPY/USD Spot Rate,Total Nonfarm Employees,Spot Crude Oil Price
Date,,,,,,,,,,,,,,
2014-10-01,5.7,237.430,0.02,0.05,2.30,0.09,3.50,2006.350,16.530,17912.079,1155000,108.03,139804,84.40
2014-11-01,5.8,236.983,0.02,0.07,2.33,0.09,3.63,2039.820,13.330,17912.079,1458000,116.30,140088,75.79
2014-12-01,5.6,236.252,0.03,0.11,2.21,0.12,3.78,2063.890,15.155,17912.079,1138000,119.32,140366,59.29
2015-01-01,5.7,234.747,0.03,0.08,1.88,0.11,3.64,2024.465,19.455,18063.529,1438000,118.25,140562,47.22
2015-02-01,5.5,235.342,0.02,0.07,1.98,0.11,3.65,2096.990,15.450,18063.529,1205000,118.76,140831,50.58
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2024-05-01,4.0,313.225,5.25,5.17,4.48,5.33,5.75,5256.815,12.920,29016.714,892000,155.87,158430,80.02
2024-06-01,4.1,313.049,5.24,5.15,4.31,5.33,5.63,5433.740,12.645,29016.714,1184000,157.86,158548,79.77
2024-07-01,4.3,313.534,5.20,5.01,4.25,5.33,5.53,5550.165,13.190,29016.714,953000,157.52,158692,81.80


# YFinance API Calls

In [93]:
sp500 = yf.download('^GSPC', start=start_date, end=end_date, interval='1mo').add_prefix('SP500 ')
nasdaq = yf.download('^IXIC', start=start_date, end=end_date, interval='1mo').add_prefix('NASDAQ ')
dji = yf.download('^DJI', start=start_date, end=end_date, interval='1mo').add_prefix('DJI ')
russ2000 = yf.download('^RUT', start=start_date, end=end_date, interval='1mo').add_prefix('RUSSELL2000 ')
ftse = yf.download('^FTSE', start=start_date, end=end_date, interval='1mo').add_prefix('FTSE ')
nikkei = yf.download('^N225', start=start_date, end=end_date, interval='1mo').add_prefix('NIKKEI ')
gold = yf.download('GLD', start=start_date, end=end_date, interval='1mo').add_prefix('GOLD ')
oil = yf.download('USO', start=start_date, end=end_date, interval='1mo').add_prefix('OIL ')
usd_index = yf.download('DX-Y.NYB', start=start_date, end=end_date, interval='1mo').add_prefix('USD INDEX ')

[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed


In [94]:
sp500_volume = sp500.drop(columns=['SP500 Low','SP500 Close','SP500 Adj Close'])

In [95]:
merged_yf_data = pd.merge(sp500_volume, nasdaq, on='Date')
merged_yf_data = pd.merge(merged_yf_data, dji, on='Date')
merged_yf_data = pd.merge(merged_yf_data, russ2000, on='Date')
merged_yf_data = pd.merge(merged_yf_data, ftse, on='Date')
merged_yf_data = pd.merge(merged_yf_data, nikkei, on='Date')
merged_yf_data = pd.merge(merged_yf_data, gold, on='Date')
merged_yf_data = pd.merge(merged_yf_data, usd_index, on='Date')
merged_yf_data = pd.merge(merged_yf_data, oil, on='Date')
merged_yf_data = pd.merge(merged_yf_data, merged_data['VIX'], on='Date')

In [96]:
merged_yf_data = merged_yf_data.drop(columns=['USD INDEX Volume'])
merged_yf_data.to_csv("VIX_YF_data.csv")


In [97]:
all_data = pd.merge(merged_data, merged_yf_data, on='Date')
all_data = all_data.drop(columns=['VIX_y'])
all_data = all_data.rename(columns={'VIX_x':'VIX'})

In [98]:
all_data.to_csv("VIX_data.csv")

In [99]:
all_data

,Unemployment Rate,CPI,Three Month T Bill Rate,Six Month T Bill Rate,10-Year Treasury Constant Maturity Rate,Fed Rate,BBB Rate,SP500 Index,VIX,GDP,...,USD INDEX High,USD INDEX Low,USD INDEX Close,USD INDEX Adj Close,OIL Open,OIL High,OIL Low,OIL Close,OIL Adj Close,OIL Volume
Date,,,,,,,,,,,,,,,,,,,,,
2014-10-01,5.7,237.430,0.02,0.05,2.30,0.09,3.50,2006.350,16.530,17912.079,...,87.129997,84.470001,86.879997,86.879997,276.799988,279.839996,241.279999,245.039993,245.039993,22051206
2014-11-01,5.8,236.983,0.02,0.07,2.33,0.09,3.63,2039.820,13.330,17912.079,...,88.440002,86.879997,88.360001,88.360001,244.080002,245.759995,203.679993,204.639999,204.639999,26087667
2014-12-01,5.6,236.252,0.03,0.11,2.21,0.12,3.78,2063.890,15.155,17912.079,...,90.330002,87.629997,90.269997,90.269997,202.479996,211.119995,158.720001,162.880005,162.880005,60335993
2015-01-01,5.7,234.747,0.03,0.08,1.88,0.11,3.64,2024.465,19.455,18063.529,...,95.480003,90.330002,94.800003,94.800003,159.759995,162.800003,130.399994,142.559998,142.559998,83013319
2015-02-01,5.5,235.342,0.02,0.07,1.98,0.11,3.65,2096.990,15.450,18063.529,...,95.360001,93.250000,95.250000,95.250000,147.199997,162.320007,140.639999,144.800003,144.800003,93894742
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2024-05-01,4.0,313.225,5.25,5.17,4.48,5.33,5.75,5256.815,12.920,29016.714,...,106.489998,104.080002,104.669998,104.669998,77.370003,78.379997,73.879997,74.820000,74.820000,63365200
2024-06-01,4.1,313.049,5.24,5.15,4.31,5.33,5.63,5433.740,12.645,29016.714,...,106.129997,103.989998,105.870003,105.870003,73.930000,80.040001,70.440002,79.589996,79.589996,41372900
2024-07-01,4.3,313.534,5.20,5.01,4.25,5.33,5.53,5550.165,13.190,29016.714,...,106.050003,103.650002,104.099998,104.099998,80.209999,82.599998,74.000000,77.739998,77.739998,49747700
